In [ ]:
# !pip install openai
# !rm /work/pi_dhruveshpate_umass_edu/project_19/aishwarya/696DS-named-entity-extraction-and-linking-for-KG-construction/code/openai/credentials.json

In [1]:
import os
import csv
import time
import json
import pandas as pd
from tqdm import tqdm

from openai import OpenAI
from dotenv import load_dotenv

In [2]:
load_dotenv("credentials.env")
api_key = os.getenv("api_key")

In [3]:
client = OpenAI(api_key=api_key)
model = "gpt-4" # gpt-3.5-turbo-instruct gpt-4

## Get Data

In [4]:
data_path = "/work/pi_dhruveshpate_umass_edu/project_19/ReDocREDPreprocessing/Re-DocRED/processed/"
file = "Re-DocRED_Processed_Dev.csv"

data = pd.read_csv(os.path.join(data_path, file))
data = data.rename(columns={'index': 'original_index'})
print(data.shape)
data.head()

(500, 4)


,original_index,Title,Text,Triplets
0,0,Willi Schneider (skeleton racer),"Wilfried "" Willi "" Schneider (born 13 March 19...",2002 Winter Olympics | start time | 2002\n2002...
1,1,Ross Alger,"Ross Patterson Alger (August 20, 1920 January...",Ross Patterson Alger | place of birth | Prelat...
2,2,Mess of Blues (Jeff Healey album),Mess of Blues is an album by Jeff Healey. It w...,Mess of Blues | publication date | 2008\nMess ...
3,3,Ramey Idriss,Ramey Idriss (11 September 1911 5 February 19...,Wet Blanket Policy | publication date | 1948\n...
4,4,ELAM (Latin American School of Medicine) Cuba,"Escuela Latinoamericana de Medicina (ELAM), fo...",Latin American School of Medicine | country | ...


## Get prompt

In [5]:
import sys
sys.path.append("../")
from openai_redocred_prompt import get_relations_prompt as get_prompt

In [6]:
def process_gtEntities(triplets):
    
    triplets = triplets.strip()
    triplets = triplets.split("\n")
    
    output = set()
    for t in triplets:
        
        if not isinstance(t,str):
            print(f"Not string - {t}")
            continue
        
        t = t.split(" | ")
        
        if len(t) != 3:
            print(f"Not len 3 - {t}")
            continue
        
        output.add(t[0])
        output.add(t[2])
        
    output = list(output)
    return output

In [ ]:
# print(data["Triplets"][idx])

In [ ]:
idx = 2
text = data["Text"][idx]

prompt = get_prompt(text, process_gtEntities(data["Triplets"][idx]))
# prompt

## Predictions

In [7]:
def get_prediction(text):
    
    prompt = get_prompt(text)
    
    start = time.time()
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt},
        ],
        temperature=0,
    )
    time_diff = time.time() - start
    
    input_len = len(prompt.replace("\n", " ").split(" "))
    
    return response, time_diff, input_len


def get_relation_prediction(idx):
    
    prompt = get_prompt(data["Text"][idx],
                        process_gtEntities(data["Triplets"][idx]))
    
    start = time.time()
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt},
        ],
        temperature=0,
    )
    time_diff = time.time() - start
    
    input_len = len(prompt.replace("\n", " ").split(" "))
    
    return response, time_diff, input_len

output_filename = "openai_relations_gt_redocred_dev_v4.csv"

In [14]:
output_data = []
debug = False

for i in tqdm(range(data.shape[0])):

    # if i != 800:
    #     continue
    
    if i == 492 or i in check_id:
        continue
    
    
    # output = get_prediction(data["Text"][i])
    output, time_diff, input_len = get_relation_prediction(i)
    
    outputText = output.choices[0].message.content
    input_tokens = output.usage.prompt_tokens
    output_tokens = output.usage.completion_tokens
    
    if debug:
        print(f"\nInput - {data['Text'][i]}")
        print(f"\nGT - {data['Triplets'][i]}")
        print(f"\noutput - {outputText}")
    
    output_data.append([data["original_index"][i], outputText, time_diff,
                       input_tokens, output_tokens])
    
    if i % 10 == 0 and i > 1:
        
        with open(output_filename, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerows(output_data)
    if debug: break
    
with open(output_filename, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(output_data)

 43%|████▎     | 217/500 [38:51<50:41, 10.75s/it]    


KeyboardInterrupt: 

In [17]:
for item in output_data:
    print(item[0])

191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216


In [ ]:
output_filename = "openai_relations_gt_redocred_dev_v6.csv"

# output_data = []
debug = False

for i in tqdm(range(data.shape[0])):

    # if i != 800:
    #     continue
    
    if i == 492 or i in check_id or i <= 216 or i > 270:
        continue
    
    
    # output = get_prediction(data["Text"][i])
    output, time_diff, input_len = get_relation_prediction(i)
    
    outputText = output.choices[0].message.content
    input_tokens = output.usage.prompt_tokens
    output_tokens = output.usage.completion_tokens
    
    if debug:
        print(f"\nInput - {data['Text'][i]}")
        print(f"\nGT - {data['Triplets'][i]}")
        print(f"\noutput - {outputText}")
    
    output_data.append([data["original_index"][i], outputText, time_diff,
                       input_tokens, output_tokens])
    
    if i % 10 == 0 and i > 1:
        
        with open(output_filename, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerows(output_data)
    if debug: break
    
with open(output_filename, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(output_data)

 49%|████▉     | 247/500 [51:27<9:05:05, 129.27s/it] 

In [ ]:
output

In [ ]:
output.usage.completion_tokens, output.usage.prompt_tokens, 1989+414

## Combine results

In [5]:
homeFolder = "/work/pi_dhruveshpate_umass_edu/project_19/aishwarya/696DS-named-entity-extraction-and-linking-for-KG-construction/code/openai"

main_output = 'openai_relations_gt_redocred_dev_all.csv'

DataPath0 = os.path.join(homeFolder, "openai_relations_gt_redocred_dev_combined.csv")
DataPath1 = os.path.join(homeFolder, "openai_relations_gt_redocred_dev_v3.csv")
DataPath2 = os.path.join(homeFolder, "openai_relations_gt_redocred_dev_v5.csv")
DataPath3 = os.path.join(homeFolder, "openai_relations_gt_redocred_dev_v6.csv")

Data0 = pd.read_csv(DataPath0, header=None)
Data1 = pd.read_csv(DataPath1, header=None)
Data2 = pd.read_csv(DataPath2, header=None)
Data3 = pd.read_csv(DataPath3, header=None)
Data = pd.concat([Data0, Data3, Data2, Data1], axis=0)

Data = Data.rename(columns={0: 'original_index', 1: 'entities', 2: 'latency',
                           3: 'input_tokens',  4: 'output_tokens'})

Data.to_csv(main_output, index=False)

print(Data.shape, len(list(set(list(Data["original_index"])))))
Data.head()

(499, 5) 499


,original_index,entities,latency,input_tokens,output_tokens
0,0,"Wilfried "" Willi "" Schneider | birth date | 13...",17.211623,1803,607
1,1,Ross Patterson Alger | date of birth | August ...,266.745473,1854,6339
2,2,Mess of Blues | performer | Jeff\nMess of Blue...,13.016327,1708,493
3,3,Ramey Idriss | date of birth | 11 September 19...,21.689985,1870,451
4,4,Escuela Latinoamericana de Medicina | country ...,171.859868,1847,6346


In [3]:
len(list(set(list(Data["original_index"]))))

499

In [12]:
check_id = list(set(list(Data["original_index"])))

In [13]:
check_id

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
